In [ ]:
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import glob

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
from pesummary.io import read as pe_read
import seaborn as sns

from thesis_utils.plotting import get_default_corner_kwargs
import corner

from thesis_utils.gw import get_cbc_parameter_labels
from thesis_utils.plotting import set_plotting, save_figure

from utils import ANALYSIS_LABELS, load_pe_summary_posterior_samples

sns.set_palette("colorblind")
set_plotting()

plt.rcParams["axes.grid"] = False

In [ ]:
event = "GW170729"
results_to_plot = ["GWTC-2", "nessai-XPHM", "nessai-XP"]

In [ ]:
parameters_to_plot = [
    # "chirp_mass",
    # "mass_ratio",
    "mass_1",
    "mass_2",
    # "a_1",
    # "a_2",
    # "tilt_1",
    # "tilt_2",
    "chi_eff",
    "chi_p",
    "ra",
    "dec",
    "theta_jn",
    "luminosity_distance",
]

In [ ]:
posterior_samples = load_pe_summary_posterior_samples(
    path="gwtc-1/comparison/",
    events=[event],
)

In [ ]:
corner_data = {}
for rtp in results_to_plot:
    res = posterior_samples[event][rtp]
    array = np.array([res[p] for p in parameters_to_plot]).T
    corner_data[rtp] = array

In [ ]:
# corner_kwargs = get_default_corner_kwargs()
# corner_kwargs.pop("color")
# corner_kwargs["fill_contours"] = False
# corner_kwargs["show_titles"] = False
# corner_kwargs["quantiles"] = None
# corner_kwargs["plot_datapoints"] = False
# corner_kwargs["plot_density"] = False
corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_density"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["show_titles"] = False
corner_kwargs.pop("fill_contours")
corner_kwargs["levels"] = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0))
corner_kwargs["quantiles"] = []
corner_kwargs["smooth"] = 0.9
corner_kwargs["bins"] = 32
fig = None

In [ ]:
fig = None
colours = ["k", "C0", "C1"]
linestyles = ["--", "-", "-"]
labels = get_cbc_parameter_labels(parameters_to_plot, units=True)
legend_elements = []
for c, ls, (key, data) in zip(colours, linestyles, corner_data.items()):
    corner_kwargs["color"] = c
    corner_kwargs["hist_kwargs"]["color"] = c
    corner_kwargs["hist_kwargs"]["ls"] = ls
    corner_kwargs["contour_kwargs"] = dict(linestyles=[ls])
    fig = corner.corner(
        data,
        fig=fig,
        labels=labels,
        **corner_kwargs,
    )
    legend_elements.append(
        Line2D([0], [0], color=c, ls=ls, label=ANALYSIS_LABELS.get(key))
    )
fig.legend(
    handles=legend_elements,
    bbox_to_anchor=(1.0, 0.9),
    loc="upper right",
    fontsize=18,
)
plt.show()
save_figure(fig, f"{event}_gwtc_2p1_comparison", f"figures/{event}")

In [ ]:
all_keys = set.intersection(
    *[set(s.dtype.names) for s in posterior_samples[event].values()]
)

In [ ]:
os.makedirs(f"figures/{event}/", exist_ok=True)
keys = set.intersection(all_keys, set(parameters_to_plot))
for key in keys:
    xlabel = get_cbc_parameter_labels(key, units=True)
    hist_kwargs = dict(
        histtype="step",
        density=True,
        bins=32,
    )
    fig = plt.figure()
    for analysis, samples in posterior_samples[event].items():
        plt.hist(
            samples[key],
            label=ANALYSIS_LABELS.get(analysis),
            **hist_kwargs,
        )
    plt.xlabel(xlabel)
    plt.legend(loc=2)
    plt.show()
    fig.savefig(f"figures/{event}/{key}.png")